# Beggs-Brill Method

In [2]:
#Imports

import numpy as np
import plotly as py
import plotly.graph_objs as go

# Initialise plotly for offline plotting
py.offline.init_notebook_mode()

In [3]:
#Gravitational constant (ft/s**2)
g = 32.174

class beggs_brill_method():
    """
    A single instance of the Beggs Brill Method for calculating the pressure drop along a pipe in a multi-phase flow regime.
    
    Calculations taken from http://cheguide.com/2015/08/beggs-brill-method/
    
    All inputs are expected in SI units, all outputs are in imperial (just to be difficult).
    
    Args:
        ID: Inner diameter (mm)
        roughness: Pipe roughness (mm)
        length: Pipe length (m)
        flow_dir: "uphill" or "downhill"
        inclination: not used, use inclination (radians) when calling acceleration_DP
        
        gas_flow_temp: Inflow temperature of gas phase (deg C)
        gas_inlet_pressure: Inflow pressure of gas phase (bar)
        gas_volumetric_flow: Inflow gas volumetric flow rate (m3/h)
        gas_density: Inflow gas density (kg/m3)
        gas_viscosity: Inflow gas viscosity(cP)
        
        liquid_volumetric_flow: Inflow liquid volumetric flow rate (m3/h)
        liquid_density: Inflow liquid density (kg/m3)
        liquid_viscosity: Inflow liquid viscosity (cP)
        
        phase_surface_tension: Gas to Liquid surface tension (dyne/cm)
        
    Usage:
        acceleration_DP(inclination):
            returns the pressure drop/unit length in PSI/ft
            where: dP/dZ = [(dP/dZ)friction + (dP/dZ)elevation]/(1 - Ek)
        
    """
    
    def __init__(self, ID, roughness, length, flow_dir, inclination, 
                 gas_flow_temp, gas_inlet_pressure, gas_volumetric_flow, gas_density, gas_viscosity,
                 liquid_volumetric_flow, liquid_density, liquid_viscosity, phase_surface_tension):
        # Flow geometries
        self.ID = ID / 25.4
        self.roughness = roughness / 25.4
        self.length = length / 0.3048
        self.flow_dir = flow_dir
        self.inclination = inclination
        
        # Gas flow properties
        self.gas_flow_temp = gas_flow_temp * 9/5 + 32
        self.gas_inlet_pressure = gas_inlet_pressure * 14.503774
        self.gas_volumetric_flow = gas_volumetric_flow / (0.3048**3) / 60
        # @ 1 atm, 0 degrees C
        self.gas_standard_volumetric_flow = (self.gas_inlet_pressure*self.gas_volumetric_flow/(self.gas_flow_temp+459.67))*(60+459.67)/14.7
        self.gas_density = gas_density / 16.018463
        self.gas_viscosity = gas_viscosity / 1488.1639
        
        # Liquid flow properties
        self.liquid_volumetric_flow = liquid_volumetric_flow * 4.4028675
        self.liquid_density = liquid_density / 16.018463
        self.liquid_viscosity = liquid_viscosity / 1488.1639
        self.phase_surface_tension = phase_surface_tension
        
        #coefficients
        self.horiz_coefficients = {
            "segregated": [0.98, 0.4846, 0.0868],
            "intermittent": [0.845, 0.5351, 0.0173],
            "distributed": [1.065, 0.5824, 0.0609]
        }
        
        self.uphill_coefficients = {
            "segregated": [0.011, -3.768, 3.539, -1.614],
            "intermittent": [2.96, 0.305, -0.4473, 0.0978],
            "distributed": [0, 0, 0, 0]
        }
        
        self.downhill_coefficients = {
            "segregated": [4.7, -0.3692, 0.1244, -0.5056],
            "intermittent": [4.7, -0.3692, 0.1244, -0.5056],
            "distributed": [4.7, -0.3692, 0.1244, -0.5056]
        }
    
    @property
    def flow_area(self):
        return np.pi*((self.ID/12)**2)/4
    
    @property
    def liquid_flowrate(self):
        metric_volume = self.liquid_volumetric_flow/4.4028675
        return (metric_volume/(0.3048**3))/3600
    
    @property 
    def gas_flowrate(self):
        return self.gas_volumetric_flow/60
    
    @property
    def liquid_superficial_velocity(self):
        return self.liquid_flowrate/self.flow_area
    
    @property
    def gas_superficial_velocity(self):
        return self.gas_flowrate/self.flow_area
    
    @property
    def liquid_weight_flux(self):
        return self.liquid_superficial_velocity*self.liquid_density
    
    @property
    def gas_weight_flux(self):
        return self.gas_superficial_velocity*self.gas_density
    
    @property
    def mixture_velocity(self):
        return self.liquid_superficial_velocity + self.gas_superficial_velocity
    
    @property
    def noslip_liquid_holdup(self):
        return self.liquid_flowrate/(self.liquid_flowrate + self.gas_flowrate)
    
    @property
    def froude_number(self):
        return (self.mixture_velocity**2) / (g*self.ID/12)
    
    @property
    def mixture_viscosity(self):
        return self.noslip_liquid_holdup*self.liquid_viscosity + (1-self.noslip_liquid_holdup) * self.gas_viscosity
    
    @property
    def mixture_density(self):
        return self.noslip_liquid_holdup*self.liquid_density + (1-self.noslip_liquid_holdup) * self.gas_density
    
    @property
    def L1(self):
        return 316*self.noslip_liquid_holdup**0.302
    
    @property
    def L2(self):
        return 0.0009252*self.noslip_liquid_holdup**-2.4684
    
    @property
    def L3(self):
        return 0.1*self.noslip_liquid_holdup**-1.4516
    
    @property
    def L4(self):
        return 0.5*self.noslip_liquid_holdup**-6.738
    
    @property
    def is_segregated(self):
        if (self.noslip_liquid_holdup < 0.01 and self.froude_number < self.L1) or (self.noslip_liquid_holdup >= 0.01 and self.froude_number < self.L2):
            return True
        else:
            return False
        
    @property
    def is_intermittent(self):
        if (0.01 <= self.noslip_liquid_holdup < 0.4 and self.L3 < self.froude_number <= self.L1) or (self.noslip_liquid_holdup >= 0.4 and self.L3 < self.froude_number <= self.L4):
            return True
        else:
            return False
        
    @property
    def is_distributed(self):
        if (self.noslip_liquid_holdup < 0.4 and self.froude_number >= self.L4) or (self.noslip_liquid_holdup >= 0.4 and self.L3 < self.froude_number <= self.L4):
            return True
        else:
            return False
        
    @property
    def is_transition(self):
        if self.L2 < self.froude_number <self.L3:
            return True
        else:
            return False
        
    @property
    def regime(self):
        if self.is_segregated:
            return "segregated"
        elif self.is_intermittent:
            return "intermittent"
        elif self.is_distributed:
            return "distributed"
        else:
            return "transition"
        
    def horiz_regime(self, regime):
        cf = self.horiz_coefficients[regime]
        return cf[0] * self.noslip_liquid_holdup**cf[1] / self.froude_number**cf[2]
    
    @property
    def liquid_velocity_number(self):
        return 1.938*self.liquid_superficial_velocity*((self.liquid_density/(self.phase_surface_tension))**0.25)
    
    def beta(self, regime, direction):
        if direction == "uphill":
            cf = self.uphill_coefficients[regime]
        else:
            cf = self.downhill_coefficients[regime]
        
        if cf[0]*(self.noslip_liquid_holdup**cf[1])*(self.liquid_velocity_number)**cf[2]*(self.froude_number)**cf[3] == 0.0:
            return 0.0
        else:
            return (1 - self.noslip_liquid_holdup)*np.log(cf[0]*(self.noslip_liquid_holdup**cf[1])*(self.liquid_velocity_number)**cf[2]*(self.froude_number)**cf[3])
    
    def correction_factor(self, regime, theta):
        return 1 + self.beta(regime, self.flow_dir)*(np.sin(1.8*theta) - (1/3)*np.sin(1.8*theta)**3)

    def horiz_liquid_holdup(self, regime):
        cf = self.horiz_coefficients[regime]
        init_el =  cf[0]*(self.noslip_liquid_holdup**cf[1])/(self.froude_number**cf[2])
        if init_el <= self.noslip_liquid_holdup:
            return self.noslip_liquid_holdup
        else:
            return init_el
    
    def deviated_liquid_holdup(self, theta, regime):
        if regime == 'transition':
            return self.transition_deviated_liquid_holdup(theta)
        else:
            return self.correction_factor(regime, theta) *  self.horiz_liquid_holdup(regime)
    
    @property
    def transition_coeff_A(self):
        return (self.L3 - self.froude_number)/(self.L3 - self.L2)
    
    @property
    def transition_coeff_B(self):
        return 1 - self.transition_coeff_A
    
    def transition_deviated_liquid_holdup(self, theta):
        return self.transition_coeff_A*self.deviated_liquid_holdup(theta, "segregated") + self.transition_coeff_B*self.deviated_liquid_holdup(theta, "intermittent")
    
    def two_phase_density(self, theta):
        regime = self.regime
        deviated_liquid_holdup = self.deviated_liquid_holdup(theta, regime)
        return self.liquid_density*deviated_liquid_holdup + self.gas_density*(1-deviated_liquid_holdup)
    
    def hydrostatic_DP(self, theta):
        return self.two_phase_density(theta)*self.length*np.sin(theta)/(144)
        
    @property
    def noslip_reynolds(self):
        return (self.ID/12)*self.mixture_density*self.mixture_velocity/self.mixture_viscosity
    
    @property
    def epsilon(self):
        return self.roughness/self.ID
    
    def y(self, theta):
        return self.noslip_liquid_holdup / (self.deviated_liquid_holdup(theta, self.regime)**2)
    
    def s(self, theta):
        
        y = self.y(theta)
        
        if y == 0:
            return 0
        elif 1 < y < 1.2:
            return np.log(2.2*y - 1.2)
        else:
            lny = np.log(y)
            return lny/(-0.0523 + 3.182 * lny - 0.8725 * (lny**2) + 0.01853 * (lny**4))
    
    def friction_factor_ratio(self, theta):
        return np.exp(self.s(theta))
    
    def fanning_friction_factor(self, theta, iterations = 18):
        
        epsilon = self.epsilon
        noslip_reynolds = self.noslip_reynolds
        f_DP = 64/self.noslip_reynolds
        
        if noslip_reynolds <= 2100:
            return f_DP/4
        else:
            for i in range(iterations):
                f_DP_new = 1/(2*np.log10(epsilon/3.7 + 2.51/(noslip_reynolds*np.sqrt(f_DP))))**2
                f_DP = f_DP_new
            return f_DP/4
                
    def two_phase_friction_factor(self, theta, iterations = 18):
        return self.friction_factor_ratio(theta)*self.fanning_friction_factor(theta, iterations)
    
    def friction_DP(self, theta, iterations = 18):
        return 2 * self.two_phase_friction_factor(theta, iterations) * (self.mixture_velocity**2) * self.mixture_density * self.length / (144 * g * (self.ID/12))
    
    def acceleration_friction_factor(self, theta):
        return self.two_phase_density(theta)*self.mixture_velocity*self.gas_superficial_velocity / (g*self.gas_inlet_pressure)
    
    def total_DP(self, theta, iterations=18):
        return self.friction_DP(theta, iterations) + self.hydrostatic_DP(theta)
    
    def acceleration_DP(self, theta, iterations = 18):
        return self.total_DP(theta, iterations) / (1 - self.acceleration_friction_factor(theta))
                                                      

In [60]:
cf = {'ID': 25.4, 'roughness': 0.00180, 'length': 100.0, 'flow_dir': 'uphill', 'inclination': 90,
     'gas_flow_temp': 10.0, 'gas_inlet_pressure': 3, 'gas_volumetric_flow': 0.7, 'gas_density': 1.225,
     'gas_viscosity': 1.81e-05, 'liquid_volumetric_flow': 2, 'liquid_density': 1000.0, 'liquid_viscosity': 8.9e-03,
     'phase_surface_tension': 74.0}

water_velocities = np.arange(0.01, 100, 0.5)
gas_velocities = np.arange(0.01, 100, 0.5)

flow_types = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
solution_velocities = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
superficial_gas_velocities = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
superficial_liquid_velocities = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))

types = {'segregated': 1, 'intermittent': 2, 'distributed': 3, 'transition': 4}

for i, wv in enumerate(water_velocities):
    for j, gv in enumerate(gas_velocities):
        cf['liquid_volumetric_flow'] = wv
        cf['gas_volumetric_flow'] = gv
        
        m = beggs_brill_method(**cf)
        solution_velocities[i, j] = m.mixture_velocity
        #superficial_gas_velocities[i, j] = m.gas_superficial_velocity
        superficial_liquid_velocities[i, j] = m.liquid_superficial_velocity
        flow_types[i, j] = types[m.regime]

In [61]:
data = [
    go.Contour(
    z = flow_types,
    y = water_velocities,
    x = gas_velocities)
]
layout = go.Layout(title = "Flow Regime Map 1 in. Tubing",
                  xaxis = dict(title="Air Flow Rate (m3/hr)"),
                  yaxis = dict(title = "Water Flow Rate (m3/hr)"))

figure = go.Figure(data = data, layout = layout)
py.offline.iplot(figure)

In [62]:
cf = {'ID': 50.8, 'roughness': 0.00180, 'length': 100.0, 'flow_dir': 'uphill', 'inclination': 90,
     'gas_flow_temp': 10.0, 'gas_inlet_pressure': 3, 'gas_volumetric_flow': 0.7, 'gas_density': 1.225,
     'gas_viscosity': 1.81e-05, 'liquid_volumetric_flow': 2, 'liquid_density': 1000.0, 'liquid_viscosity': 8.9e-03,
     'phase_surface_tension': 74.0}

water_velocities = np.arange(0.01, 100, 0.5)
gas_velocities = np.arange(0.01, 100, 0.5)

flow_types = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
solution_velocities = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))

types = {'segregated': 1, 'intermittent': 2, 'distributed': 3, 'transition': 4}

for i, wv in enumerate(water_velocities):
    for j, gv in enumerate(gas_velocities):
        cf['liquid_volumetric_flow'] = wv
        cf['gas_volumetric_flow'] = gv
        
        m = beggs_brill_method(**cf)
        solution_velocities[i, j] = m.mixture_velocity
        flow_types[i, j] = types[m.regime]

In [63]:
data = [
    go.Contour(
    z = flow_types,
    x = gas_velocities,
    y = water_velocities)
]

layout = go.Layout(title = "Flow Regime Map 2 in. Tubing",
                  yaxis = dict(title="Air Flow Rate (m3/hr)"),
                  xaxis = dict(title = "Water Flow Rate (m3/hr)"))

figure = go.Figure(data = data, layout = layout)
py.offline.iplot(figure)

In [64]:
cf = {'ID': 76.2, 'roughness': 0.00180, 'length': 100.0, 'flow_dir': 'uphill', 'inclination': 90,
     'gas_flow_temp': 10.0, 'gas_inlet_pressure': 3, 'gas_volumetric_flow': 0.7, 'gas_density': 1.225,
     'gas_viscosity': 1.81e-05, 'liquid_volumetric_flow': 2, 'liquid_density': 1000.0, 'liquid_viscosity': 8.9e-03,
     'phase_surface_tension': 74.0}

water_velocities = np.arange(0.01, 100, 0.5)
gas_velocities = np.arange(0.01, 100, 0.5)

flow_types = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
solution_velocities = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
superficial_gas_velocities = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
superficial_liquid_velocities = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))

types = {'segregated': 1, 'intermittent': 2, 'distributed': 3, 'transition': 4}

for i, wv in enumerate(water_velocities):
    for j, gv in enumerate(gas_velocities):
        cf['liquid_volumetric_flow'] = wv
        cf['gas_volumetric_flow'] = gv
        
        m = beggs_brill_method(**cf)
        solution_velocities[i, j] = m.mixture_velocity
        superficial_gas_velocities[i, j] = m.gas_superficial_velocity
        superficial_liquid_velocities[i, j] = m.liquid_superficial_velocity
        flow_types[i, j] = types[m.regime]

In [65]:
data = [
    go.Contour(
    z = flow_types,
    x = gas_velocities,
    y = water_velocities)
]
layout = go.Layout(title = "Flow Regime Map 3 in. Tubing",
                  yaxis = dict(title="Air Flow Rate (m3/hr)"),
                  xaxis = dict(title = "Water Flow Rate (m3/hr)"))

figure = go.Figure(data = data, layout = layout)
py.offline.iplot(figure)

In [66]:
cf = {'ID': 25.4, 'roughness': 0.00180, 'length': 100.0, 'flow_dir': 'uphill', 'inclination': 90,
     'gas_flow_temp': 10.0, 'gas_inlet_pressure': 3, 'gas_volumetric_flow': 0.7, 'gas_density': 1.225,
     'gas_viscosity': 1.81e-05, 'liquid_volumetric_flow': 2, 'liquid_density': 1000.0, 'liquid_viscosity': 8.9e-03,
     'phase_surface_tension': 74.0}

water_velocities = np.arange(0.01, 100, 1.0)
gas_velocities = np.arange(0.01, 100, 1.0)

deltaP = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
regime = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))

types = {'segregated': 1, 'intermittent': 2, 'distributed': 3, 'transition': 4}

for i, wv in enumerate(water_velocities):
    for j, gv in enumerate(gas_velocities):
        cf['liquid_volumetric_flow'] = wv
        cf['gas_volumetric_flow'] = gv
        
        m = beggs_brill_method(**cf)
        deltaP[i, j] = m.acceleration_DP(0.00)
        #superficial_gas_velocities[i, j] = m.gas_superficial_velocity
        regime[i, j] = types[m.regime]

In [67]:
data = [
    go.Contour(
    z = deltaP * 6.89476,
    x = gas_velocities,
    y = water_velocities,
    autocontour = False,
    contours = dict(
        end = 100,
        start = -1000,
        size = 100,
        )
    )
]
layout = go.Layout(title = "Delta P (kPa) ",
                  xaxis = dict(title="Air Flow Rate (m3/hr)"),
                  yaxis = dict(title = "Water Flow Rate (m3/hr)"))

figure = go.Figure(data = data, layout = layout)
py.offline.iplot(figure)

In [68]:
cf = {'ID': 25.4, 'roughness': 0.00180, 'length': 100.0, 'flow_dir': 'uphill', 'inclination': 90,
     'gas_flow_temp': 10.0, 'gas_inlet_pressure': 3, 'gas_volumetric_flow': 0.7, 'gas_density': 1.225,
     'gas_viscosity': 1.81e-05, 'liquid_volumetric_flow': 2, 'liquid_density': 1000.0, 'liquid_viscosity': 8.9e-03,
     'phase_surface_tension': 74.0}

water_velocities = np.arange(0.01, 100, 1.0)
gas_velocities = np.arange(0.01, 100, 1.0)

deltaP = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))
regime = np.empty((water_velocities.shape[0], gas_velocities.shape[0]))

types = {'segregated': 1, 'intermittent': 2, 'distributed': 3, 'transition': 4}

for i, wv in enumerate(water_velocities):
    for j, gv in enumerate(gas_velocities):
        cf['liquid_volumetric_flow'] = wv
        cf['gas_volumetric_flow'] = gv
        
        m = beggs_brill_method(**cf)
        deltaP[i, j] = m.acceleration_DP(3.14159 / 2)
        #superficial_gas_velocities[i, j] = m.gas_superficial_velocity
        regime[i, j] = types[m.regime]

In [69]:
data = [
    go.Contour(
    z = deltaP * 6.89476,
    x = gas_velocities,
    y = water_velocities,
    autocontour = False,
    contours = dict(
        end = 100,
        start = -1000,
        size = 100,
        )
    )
]
layout = go.Layout(title = "Delta P (kPa) ",
                  xaxis = dict(title="Air Flow Rate (m3/hr)"),
                  yaxis = dict(title = "Water Flow Rate (m3/hr)"))

figure = go.Figure(data = data, layout = layout)
py.offline.iplot(figure)